# Ejercicio 11

Se busca predecir si el tipo de fármaco que se debe administrar a un paciente afectado de rinitis alérgica es el habitual o no. Se dispone de información correspondiente a las historias clínicas de pacientes atendidos previamente


## Inciso a
Numerice los atributos nominales y utilice el archivo `drugs_train.csv` para entrenar un modelo con 5 neuronas que sea capaz de predecir el tipo de fármaco que se debe administrar a un paciente afectado de rinitis alérgica.

In [23]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from ClassRNMulticlase import RNMulticlase
DATA_PATH='./Data/'
# Cargar el archivo de entrenamiento
train_data = pd.read_csv(DATA_PATH + 'drugs_train.csv')

train_data['Sex']=train_data['Sex'].replace({'M':0, 'F':1})
train_data['BP']=train_data['BP'].replace({'LOW':0,'NORMAL':1,'HIGH':2})
train_data['Cholesterol']=train_data['Cholesterol'].replace({'NORMAL':0,'HIGH':1})

scaler = StandardScaler()
train_data[['Age', 'Na', 'K']] = scaler.fit_transform(train_data[['Age', 'Na', 'K']])

X=np.array(train_data.drop(columns='Drug'))
T=train_data['Drug']
T = np.array(pd.get_dummies(T)*1)

rnm=RNMulticlase(0.01,1000,1e-9,'sigmoid','ECM')

rnm.fit(X,T)

predicciones = rnm.predict(X)
accuracy = np.mean(predicciones == np.argmax(T, axis=1))
print(f"Accuracy: {accuracy:.2f}")


/tmp/ipykernel_130942/2215239812.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_data['Sex']=train_data['Sex'].replace({'M':0, 'F':1})
/tmp/ipykernel_130942/2215239812.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_data['BP']=train_data['BP'].replace({'LOW':0,'NORMAL':1,'HIGH':2})
/tmp/ipykernel_130942/2215239812.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-i

Accuracy: 0.83



## Inciso b
Luego utilice el archivo `drug_test.csv` para medir la calidad del modelo.

In [25]:
test_data = pd.read_csv(DATA_PATH + 'drugs_test.csv')

test_data['Sex']=test_data['Sex'].replace({'M':0, 'F':1})
test_data['BP']=test_data['BP'].replace({'LOW':0,'NORMAL':1,'HIGH':2})
test_data['Cholesterol']=test_data['Cholesterol'].replace({'NORMAL':0,'HIGH':1})

scaler = StandardScaler()
test_data[['Age', 'Na', 'K']] = scaler.fit_transform(test_data[['Age', 'Na', 'K']])

test_X=np.array(test_data.drop(columns='Drug'))
test_T=test_data['Drug']
test_T = np.array(pd.get_dummies(test_T)*1)




test_T = pd.get_dummies(test_data["Drug"]).astype(int).to_numpy()
test_X = test_data.drop(columns=["Drug"])

test_Y = rnm.predict(test_X)
true_Y = np.argmax(test_T, axis=1)
accuracy = np.mean(test_Y == true_Y)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.78


/tmp/ipykernel_130942/4226345011.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_data['Sex']=test_data['Sex'].replace({'M':0, 'F':1})
/tmp/ipykernel_130942/4226345011.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_data['BP']=test_data['BP'].replace({'LOW':0,'NORMAL':1,'HIGH':2})
/tmp/ipykernel_130942/4226345011.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to t

# Ejercicio 12

Utilice una red neuronal formada por una única capa de salida de 3 neuronas para clasificar las muestras de
vino del archivo `Vinos.csv` descripto en el ejercicio anterior.

Realice 30 ejecuciones independientes utilizando el 60% y 80% de los ejemplos como entrenamiento y el
resto como testeo. 

Utilice un máximo de 400 iteraciones y velocidades de aprendizaje 0.1, 0.2 y 0.3.

In [29]:
from sklearn.model_selection import train_test_split

def test_wine_nn(data, alphas=[0.1, 0.2, 0.3], train_sizes=[0.6, 0.8], n_runs=30, FUN='sigmoid', COSTO='ECM'):
  results = []
  for alpha in alphas:
    for train_size in train_sizes:
      accs = []
      for _ in range(n_runs):
        # Split data
        X = data.drop(columns='Class')
        T = pd.get_dummies(data['Class']).astype(int).to_numpy()
        X_train, X_test, T_train, T_test = train_test_split(X, T, train_size=train_size, stratify=T)
        
        # Standardize
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        # Train model
        rnm = RNMulticlase(alpha, 400, 1e-9, FUN=FUN, COSTO=COSTO)
        rnm.fit(X_train_scaled, T_train)
        
        # Test
        pred = rnm.predict(X_test_scaled)
        true = np.argmax(T_test, axis=1)
        acc = np.mean(pred == true)
        accs.append(acc)
      results.append({
        'alpha': alpha,
        'train_size': train_size,
        'mean_acc': np.mean(accs),
        'std_acc': np.std(accs)
      })
  return results

data=pd.read_csv(DATA_PATH+'Vinos.csv', sep=';')


## Inciso A

Función de activación ‘sigmoid’ y función de costo ‘ECM’ (error cuadrático medio).

In [30]:
results_sig_ecm=test_wine_nn(data, FUN='sigmoid', COSTO='ECM')
for res in results_sig_ecm:
  print(f"Alpha: {res['alpha']}, Train size: {res['train_size']}, Mean accuracy: {res['mean_acc']:.2f}, Std: {res['std_acc']:.2f}")

Alpha: 0.1, Train size: 0.6, Mean accuracy: 0.94, Std: 0.02
Alpha: 0.1, Train size: 0.8, Mean accuracy: 0.95, Std: 0.04
Alpha: 0.2, Train size: 0.6, Mean accuracy: 0.93, Std: 0.03
Alpha: 0.2, Train size: 0.8, Mean accuracy: 0.93, Std: 0.04
Alpha: 0.3, Train size: 0.6, Mean accuracy: 0.94, Std: 0.02
Alpha: 0.3, Train size: 0.8, Mean accuracy: 0.94, Std: 0.04


## Inciso B

Función de activación ‘sigmoid’ y función de costo ‘EC_binaria’ (entropía cruzada binaria)

In [31]:
results_sig_ecb=test_wine_nn(data, FUN='sigmoid', COSTO='EC_binaria')
for res in results_sig_ecb:
  print(f"Alpha: {res['alpha']}, Train size: {res['train_size']}, Mean accuracy: {res['mean_acc']:.2f}, Std: {res['std_acc']:.2f}")

Alpha: 0.1, Train size: 0.6, Mean accuracy: 0.93, Std: 0.02
Alpha: 0.1, Train size: 0.8, Mean accuracy: 0.95, Std: 0.03
Alpha: 0.2, Train size: 0.6, Mean accuracy: 0.94, Std: 0.02
Alpha: 0.2, Train size: 0.8, Mean accuracy: 0.94, Std: 0.03
Alpha: 0.3, Train size: 0.6, Mean accuracy: 0.93, Std: 0.02
Alpha: 0.3, Train size: 0.8, Mean accuracy: 0.94, Std: 0.03


## Inciso C

Función de activación ‘tanh’ y función de costo ‘ECM’ (error cuadrático medio)

In [32]:
results_tanh_ecm=test_wine_nn(data, FUN='tanh', COSTO='ECM')
for res in results_tanh_ecm:
  print(f"Alpha: {res['alpha']}, Train size: {res['train_size']}, Mean accuracy: {res['mean_acc']:.2f}, Std: {res['std_acc']:.2f}")

Alpha: 0.1, Train size: 0.6, Mean accuracy: 0.52, Std: 0.06
Alpha: 0.1, Train size: 0.8, Mean accuracy: 0.52, Std: 0.08
Alpha: 0.2, Train size: 0.6, Mean accuracy: 0.53, Std: 0.09
Alpha: 0.2, Train size: 0.8, Mean accuracy: 0.56, Std: 0.11
Alpha: 0.3, Train size: 0.6, Mean accuracy: 0.58, Std: 0.11
Alpha: 0.3, Train size: 0.8, Mean accuracy: 0.59, Std: 0.11


## Inciso D

Capa ‘Softmax’ y función de costo ‘EC’ (entropía cruzada)

In [33]:
results_sig_ecm=test_wine_nn(data, FUN='softmax', COSTO='EC')
for res in results_sig_ecm:
  print(f"Alpha: {res['alpha']}, Train size: {res['train_size']}, Mean accuracy: {res['mean_acc']:.2f}, Std: {res['std_acc']:.2f}")

Alpha: 0.1, Train size: 0.6, Mean accuracy: 0.92, Std: 0.03
Alpha: 0.1, Train size: 0.8, Mean accuracy: 0.93, Std: 0.04
Alpha: 0.2, Train size: 0.6, Mean accuracy: 0.92, Std: 0.03
Alpha: 0.2, Train size: 0.8, Mean accuracy: 0.91, Std: 0.04
Alpha: 0.3, Train size: 0.6, Mean accuracy: 0.91, Std: 0.04
Alpha: 0.3, Train size: 0.8, Mean accuracy: 0.93, Std: 0.04
